In [2]:
import numpy as np
import pandas as pd
import re
import os
import pickle
import datetime as dt
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
from tqdm import tqdm
from konlpy.tag import Twitter

mallet_path = 'C:\\Users\\hufss\\mallet-2.0.8\\bin\\mallet'

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

np.random.seed(10)


In [3]:
#Corpus 만드는 함수
def make_corpus(text_col):
    from gensim import corpora
    
    dictionary = corpora.Dictionary(text_col)
    dictionary.filter_extremes(no_below = 20)
    corpus = [dictionary.doc2bow(text) for text in text_col]
    return dictionary, corpus

In [4]:
#LDA 모델 훈련시키는 함수
def make_LDA(corpus, dictionary, num_topics, num_words):
    import gensim
    from gensim import corpora, models
    NUM_TOPICS = num_topics
    ldamallet = models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=NUM_TOPICS,
                                          id2word=dictionary,random_seed = 10)
    topics = ldamallet.print_topics(num_words = num_words)

    return ldamallet, topics

In [5]:
#뉴스 데이터를 가져옴

news_data = pd.read_pickle('news_ready_for_LDA.pkl')

#데이터에서 가져오기 및 합치는 중 생긴 오류 값들을 제거함
news_data.drop(news_data.loc[news_data['date']=='CHIEF EXECUTIVE'].index, inplace=True)
news_data.drop(news_data.loc[news_data['date']=='date'].index, inplace=True)

#date 형식을 datetime으로 변환
news_data['date'] = pd.to_datetime(news_data['date'], format = "%Y.%m.%d.")

news_data

,Unnamed: 0,date,title,contents
0,24,2020-07-19,"[사회, 공헌, 물품, 기부, 캠페인, 행사, 유통업, 사회, 공헌]",리조이스 캠페인은 여성 우울증 예방과 인식 개선에 기여하는 기업의 사회적 책임(CS...
1,62,2020-11-26,"[사회, 공헌, 디자인, 재능, 기부, 통해, 골목, 상권, 활기, 꿈꾸다]",어워드 &amp; CSR 필름페스티벌'에서 ‘우리 가게 전담 디자이너’ 사업이 시작...
2,86,2020-04-10,"[특별, 기획, 여파, 노블레스, 오블리주, 기업, 기부]",코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
3,110,2020-10-16,"[식량, 맞다, 식품, 업계, 기부, 절약, 캠페인, 펼치다]",기업사회공헌(CSR) 전문 스토어다. 아이배냇은 앞서 지난 4월에도 코로나19 극복...
4,134,2020-04-10,"[특별, 기획, 여파, 노블레스, 오블리주, 기업, 기부]",코로나19 발발 직후 전국재해구호협회에 10억원을 기부한 LG그룹은 지주사 LG를 ...
...,...,...,...,...
37933,188658,2020-12-05,"[서산시, 부석면, 희망, 나눔, 캠페인, 행사, 개최]","20포대, 현대건설에서 쌀10kg 60포대를 어려운 이웃을 위해 사용해달라고 기부했..."
37934,188786,2020-10-23,"[망고스틴, 디지털, 캠페인, 전문, 도약]","머스트잇, 동아오츠카, 신라면세점, 현대건설, 토스 등 다양한 브랜드의 디지털 캠페..."
37936,192696,2020-01-16,"[선물, 세트, 상품권, 전달, 재래시장, 장보기, 캠페인]","이어 박종성 부사장은 ""현대제철은 앞으로도 지역민을 위한 지속적인 나눔 실천을 통해..."
37937,192704,2020-01-16,"[선물, 가구, 전달, 재래시장, 활성화, 캠페인]","현대제철 임직원 봉사단은 5만 원 상당의 선물세트 400개와 전통시장 상품권, 당진..."


In [6]:
#dictionary, corpus 생성

dictionary, corpus = make_corpus(news_data['title'])
texts = news_data['title']

In [7]:
##num_topics, num_words 정하기
Num_Topics = 8
Num_Words = 10

In [8]:
##def 실행 - ldamodel, topic 생성
ldamodel, topics = make_LDA(corpus, dictionary, Num_Topics, Num_Words)

In [12]:
####LDA모델이 잘 만들어졌는지 확인
for topic in tqdm(topics):
    print('{}번째 Topic'.format(topic[0]), topic[1])

100%|██████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 7906.32it/s]

0번째 Topic 0.054*"사랑" + 0.053*"이웃" + 0.051*"계층" + 0.049*"성금" + 0.046*"취약" + 0.045*"기탁" + 0.044*"억원" + 0.041*"전달" + 0.037*"나눔" + 0.028*"봉사"
1번째 Topic 0.171*"상생" + 0.051*"중소기업" + 0.046*"기업" + 0.042*"최우수" + 0.040*"협약" + 0.033*"지수" + 0.025*"개사" + 0.021*"공정" + 0.020*"사오다" + 0.019*"기술"
2번째 Topic 0.112*"기부" + 0.052*"극복" + 0.052*"지원" + 0.045*"캠페인" + 0.032*"위해" + 0.024*"나서다" + 0.024*"직원" + 0.023*"진행" + 0.018*"봉사활동" + 0.016*"피해"
3번째 Topic 0.059*"건설" + 0.052*"사망" + 0.048*"산업" + 0.047*"지원" + 0.031*"사고" + 0.029*"대기업" + 0.023*"거래" + 0.020*"근로자" + 0.018*"내부" + 0.018*"현장"
4번째 Topic 0.111*"과징금" + 0.066*"개인정보" + 0.062*"유출" + 0.043*"담합" + 0.033*"만원" + 0.030*"입찰" + 0.024*"부과" + 0.021*"소송" + 0.018*"업체" + 0.018*"운송"
5번째 Topic 0.196*"사회" + 0.116*"지역" + 0.106*"공헌" + 0.045*"협약" + 0.036*"체결" + 0.033*"사업" + 0.030*"업무" + 0.029*"활동" + 0.029*"기업" + 0.021*"공사"
6번째 Topic 0.088*"공정위" + 0.087*"지원" + 0.042*"불공정" + 0.039*"제재" + 0.034*"고발" + 0.021*"부당" + 0.020*"열사" + 0.019*"검찰" + 0.017*"몰다" + 0.015*"하도급"
7번째 Topic 0.164*"협

In [15]:
pd.DataFrame(topics)

,0,1
0,0,"0.054*""사랑"" + 0.053*""이웃"" + 0.051*""계층"" + 0.049*""..."
1,1,"0.171*""상생"" + 0.051*""중소기업"" + 0.046*""기업"" + 0.042..."
2,2,"0.112*""기부"" + 0.052*""극복"" + 0.052*""지원"" + 0.045*""..."
3,3,"0.059*""건설"" + 0.052*""사망"" + 0.048*""산업"" + 0.047*""..."
4,4,"0.111*""과징금"" + 0.066*""개인정보"" + 0.062*""유출"" + 0.04..."
5,5,"0.196*""사회"" + 0.116*""지역"" + 0.106*""공헌"" + 0.045*""..."
6,6,"0.088*""공정위"" + 0.087*""지원"" + 0.042*""불공정"" + 0.039..."
7,7,"0.164*""협력"" + 0.105*""지원"" + 0.042*""대금"" + 0.042*""..."


In [10]:
pickle.dump(ldamodel, open('lda_model.pkl', 'wb'))


In [11]:
pickle.dump(dictionary, open('dictionary.pkl', 'wb'))